In [1]:
import numpy as np
import os
import shutil
import csv
import pandas as pd
import gzip

In [2]:
# gene expression paths
dataPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/gdc_download_FPKM_832' #zipped file path
dataNewFolder = '/Users/ursulawidocki/Desktop/BreCaNet/Data/processedTCGA_832' # folder for unzipped
#PANDAexpPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/PANDAinput/PANDAgeneExp_832.txt' # final exp file

In [3]:
# Unzips the expression data dn creates a folder (processedTCGA_832) of the unzipped files 
# Gets the codes of the patient folders and expression files

folderIDs = list() # codes of the patient IDs on the folder
indivList = list() # codes of the patients' expression file

for fileName in os.listdir(dataPath):
    if fileName == '.DS_Store' or fileName == 'MANIFEST.txt':
        continue
        
    else:
        folderIDs.append(fileName)
        
    newPath = dataPath + '/' + fileName
    
    # goes into the individual folder
    for indivName in os.listdir(newPath):
        if (indivName == '.DS_Store') or (indivName == 'MANIFEST.txt') or (indivName == 'annotations.txt'):
            continue
        filePath = newPath + '/' + indivName
        
        fileEnding = indivName[-3] + indivName[-2] + indivName[-1]
        if(fileEnding == '.gz'):
            with gzip.open(filePath, 'rb') as f_in:
                newFilePath = dataNewFolder + '/' + indivName[0:-3]
                indivList.append(indivName[0:-3])
                
                with open(newFilePath, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)

In [4]:
## Makes file of folder names

folderIDPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/foldIDList_FPKM_832.txt'
folderFile = open(folderIDPath, "w+")
for i in range(0, 832):
    newLineFolder = folderIDs[i] + "\n"
    
    folderFile.write(newLineFolder)
    
folderFile.close()

In [10]:
## Gets folder codes from expression data
folderIDPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/foldIDList_FPKM_832.txt'
folders = pd.read_csv(folderIDPath, delimiter = '\t', header = None)
folders_list = list(folders[0])

In [5]:
## Gets file_id and case_id

metaPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/metadata.cart.FPKM_832.json'

file_id = list()
f = 0
case_id = list()
c = 0
with open(metaPath, "r") as file:
    for line in file.readlines():
        
        if "file_id" in line:
            item = line.strip().split("\"")[3]
            f = f + 1
            
            if(f%2 == 0):
                file_id.append(item)
                
        if("case_id" in line) and (f%2 == 0):
            case = line.strip().split("\"")[3]
            c = c + 1
            case_id.append(case)

print(len(file_id))
print(len(case_id))
#print(case_id[0:5])

832
832


In [7]:
## Reads in PAM50 labels

subtype_path = "/Users/ursulawidocki/Desktop/BreCaNet/Code/TCGA_subtypes.txt"
subtypes = pd.read_csv(subtype_path, delimiter = " ")
subtypes.head()

pt_barcodes_list = list(subtypes['brca_subtypes.patient'])
subtype_list = list(subtypes['brca_subtypes.BRCA_Subtype_PAM50'])

print(len(pt_barcodes_list))
print(len(subtype_list))

1087
1087


In [8]:
## Gets uuid and barcodes of the patients from clinical data

# reads in clinical information to link patients with their barcode
clinicalPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/gdc_download_clinical_832/8162d394-8b64-4da2-9f5b-d164c54b9608/nationwidechildrens.org_clinical_patient_brca.txt'
clinical_df = pd.read_csv(clinicalPath, delimiter = '\t')

# removes rows that are not necessary to match data
clinical_df = clinical_df.drop(clinical_df.index[0])
clinical_df = clinical_df.drop(clinical_df.index[0])
clinical_df.head()
codes_df = clinical_df[['bcr_patient_uuid', 'bcr_patient_barcode']]
#codes_df.head()

uuid_list = list(codes_df['bcr_patient_uuid'])
barcode_list = list(codes_df['bcr_patient_barcode'])

print(len(uuid_list))
print(len(barcode_list))

1097
1097


### Matching

In [11]:
## First, match the folder with the file_ids to get the corresponding cases from the metadata

match_cases = list()

for folder in folders_list:
    for file in file_id:
        if folder == file:
            ind = file_id.index(file)
            match_cases.append(case_id[ind])
            break

print(len(match_cases)) # should be equivalent to number of samples

832


In [ ]:
# Deletes variables no longer needed to save memory


In [12]:
## Matches case_ids from metadata with uuid and patient barcodes from clinical data

match_barcodes = list()

for case in match_cases:
    for ptID in uuid_list:
        if case == ptID.lower():
            ind = uuid_list.index(ptID)
            match_barcodes.append(barcode_list[ind])
            break
            
print(len(match_barcodes))

832
